In [27]:
import requests
import json
import numpy as np

### Hosting llama model using Ollama

In [2]:
OLLAMA_URL = "http://localhost:11434/api/generate"

Define the model

In [3]:
import requests

class OllamaLLM:
    def __init__(self, model_name, url="http://localhost:11434/api/generate", params=None):
        """
        Initialize Ollama LLM with a specific model and API endpoint.
        
        Parameters:
        - model_name (str): Name of the Ollama model (e.g., "llama3.1", "mistral").
        - url (str): URL for the Ollama server (default is localhost).
        - params (dict): Additional parameters for the model (optional).
        """
        self.url = url
        self.model_name = model_name
        self.params = params or {}

    def generate(self, prompt, stream=False):
        """
        Generate a response from the LLM.

        Parameters:
        - prompt (str): Input prompt to send to the model.
        - stream (bool): Enable streaming (default: False).

        Returns:
        - str: The generated response.
        """
        data = {
            "model": self.model_name,
            "prompt": prompt,
            "stream": stream,
        }
        data.update(self.params)

        # Send a POST request to Ollama API
        response = requests.post(self.url, json=data)

        # Check for errors
        if response.status_code == 200:
            result = response.json()
            return result['response']
        else:
            raise Exception(f"Error: {response.status_code}, {response.text}")

    def bind(self, **kwargs):
        """
        Bind additional parameters to the model.

        Parameters:
        - kwargs: Key-value pairs of additional model parameters.

        Returns:
        - OllamaLLM: A new instance with updated parameters.
        """
        # Create a new instance with updated parameters
        new_params = self.params.copy()
        new_params.update(kwargs)
        return OllamaLLM(model_name=self.model_name, url=self.url, params=new_params)
    

llm_chat = OllamaLLM(
    model_name="llama3.1",  
    params={"temperature": 0.7, "top_p": 0.9} 
)

# Example usage
response = llm_chat.generate("What is the length of the word 'incomprehensibility'")  # Calling with a prompt
print(response)



The word "incomprehensibility" has 17 letters: i-n-c-o-m-p-r-e-h-e-n-s-i-b-i-l-i-t-y.


### Prompt template prep and tools (Not being used right now)

In [4]:
system_prompt = """Below are a series of dialogues between various people and an AI assistant. The AI tries to be helpful, polite, honest, sophisticated, emotionally aware, and humble-but-knowledgeable. The assistant prioritizes caution over usefulness, refusing to answer questions that it considers unsafe, immoral, unethical or dangerous. 

AI assistant has access to the following tools:

{tools}

Use a JSON blob to specify a tool by providing an action key (tool name) and an action input key (tool input). 

Use only valid "action" values: "Final Answer" or {tool_names} and provide only one action per $JSON_BLOB. Use "action" value "Final Answer" if you know what to respond. Provide only ONE action per $JSON_BLOB, as shown:
```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```
Follow this format:
Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{{
  "action": "Final Answer",
  "action_input": "Final response to human"
}}

Reminder to ALWAYS respond with a valid JSON blob of a single action. Use tools only if necessary. Format is Action:```$JSON_BLOB```then Observation
"""

In [5]:
user_input = '{input}\n{agent_scratchpad}\n(reminder to respond in a JSON blob no matter what and use tools only if necessary)'

In [6]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import MessagesPlaceholder

Create structured prompt template

In [7]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", system_prompt
        ),
        MessagesPlaceholder(variable_name="chat_history", optional=True),
        ("user", user_input),
    ]
)

In [8]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tool_names', 'tools'], input_types={}, partial_variables={}, template='Below are a series of dialogues between various people and an AI assistant. The AI tries to be helpful, polite, honest, sophisticated, emotionally aware, and humble-but-knowledgeable. The assistant prioritizes caution over usefulness, refusing to answer questions that it considers unsafe, immoral, unethical or dangerous. \n\nAI assistant has access to the following tools:\n\n{tools}\n\nUse a JSON blob to specify a tool by providing an action key (tool name) and an action input key (tool input). \n\nUse only valid "action" values: "Final Answer" or {tool_names} and provide only one action per $JSON_BLOB. Use "action" value "Final Answer" if you know what to respond. Provide only ONE action per $JSON_BLOB, as shown:\n```\n{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}\n```\nFollow this format:\nQuestion: input question to answer\nThought: 

### Prepare Tools

In [9]:
from langchain.agents import tool
from langchain.tools import StructuredTool

In [ ]:
# Define tools
@tool
def get_word_length(word: str) -> int:
    """Return the length of the given word."""
    return len(word)

@tool
def remove_punctuations(text: str) -> str:
    """Remove all punctuation symbols from the text."""
    new_string = re.sub(r'[^\w\s]', '', text) 
    return new_string

@tool
def get_mean_from_sequence(sequence: str) -> float:
    """Return the mean value from the sequence of numbers."""
    return float(np.mean(np.array(sequence.split(','), dtype=float)))

# Structured tools
def get_normalize_vector(vector: list[float]) -> str:
    """Return a normalized `vector`."""
    return str(list(vector / np.sqrt(np.sum(np.array(vector)**2))))

get_normalize_vector_structured = StructuredTool.from_function(
    func=get_normalize_vector,
    name="get_normalize_vector",
    description="Useful when you want to normalize a vector. Input should be a list of float numbers.",
)

def get_gcd(a: int, b: int) -> int:
    """Return the greatest common divisor of the two integers."""
    return math.gcd(a, b)

get_gcd_structured = StructuredTool.from_function(
    func=get_gcd,
    name="get_gcd",
    description="Useful when you want to find greatest common divisor of the two integers.",
)


### Initialize an Agent

In [48]:
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.tools import StructuredTool, tool
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms.ollama import Ollama
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import MessagesPlaceholder
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain.agents import create_react_agent


import requests
import numpy as np
import re
import math

# Collect all tools
tools = [
    get_word_length, 
    get_mean_from_sequence, 
    remove_punctuations, 
    get_normalize_vector_structured, 
    get_gcd_structured
]

# Create a more direct system prompt that works better with Llama models
system_prompt = """You are a helpful AI assistant with access to tools.
You must ALWAYS use one of the provided tools to answer user queries.
NEVER provide answers directly - always select and use the appropriate tool.

To use a tool, you MUST use the following format:
Action: the_tool_name
Action Input: {{"parameter_name": "parameter_value"}}

For example:
Action: get_word_length
Action Input: {{"word": "example"}}

Available tools: get_word_length, get_mean_from_sequence, remove_punctuations, get_normalize_vector_structured, get_gcd_structured

After you receive the tool's response, provide a final answer based on the tool's output.
"""

# Create a cleaner template with tool_names included
template = """You are a helpful AI assistant that can use tools to answer questions.

TOOLS:
------
You have access to the following tools:
{tool_names}

{tools}

TOOL USAGE:
-----------
To use a tool, follow this format exactly:
Thought: I need to find out something
Action: The tool name
Action Input: The input to the tool

When you receive the observation from the tool, you can continue with:
Thought: I've seen the result
Action: Another tool if needed
Action Input: Another input

When you're ready to give the final answer:
Thought: I know the answer
Final Answer: Your final answer here

RULES:
------
- ALWAYS use a tool before giving a final answer
- NEVER try to answer directly without using a tool
- FORMAT your responses exactly as shown above
- NEVER include both Action and Final Answer in the same step

Begin!

Question: {input}
{agent_scratchpad}
"""

from langchain.prompts import PromptTemplate
prompt = PromptTemplate(
    template=template,
    input_variables=["input", "chat_history", "agent_scratchpad", "tools", "tool_names"]
)


# Connect to Ollama with lower temperature for more consistent output
llm = ChatOllama(
    model="llama3.1",
    temperature=0.1,  # Lower temperature for more consistent outputs
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
)

# Create the agent using OpenAI functions format which works better with Llama models
agent = create_react_agent(llm, tools, prompt)

# Create the agent executor with more iterations and better error handling
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=3,  # Reduce to simplify debugging
    early_stopping_method="force",
    return_intermediate_steps=True
)

def run_agent(query, chat_history=None):
    if chat_history is None:
        chat_history = []

    try:
        # Run the agent
        result = agent_executor.invoke({
            "input": query,
            "chat_history": chat_history
        })
        print(f"Tools Available: {[tool.name for tool in tools]}")

        print("\nRaw result:", result)  # Debugging output

        # Extract tool usage information
        tool_usage_details = ""
        if "intermediate_steps" in result:
            for step in result["intermediate_steps"]:
                if isinstance(step, tuple) and len(step) == 2:
                    tool_name = step[0].tool
                    tool_input = step[0].tool_input
                    tool_output = step[1]
                    tool_usage_details += f"\n🛠 **Tool Used:** {tool_name}\n   🔹 **Input:** {tool_input}\n   🔹 **Output:** {tool_output}\n"

        # Extract and clean output
        output = result.get("output", str(result)).strip()

        # Append tool usage details to output
        if tool_usage_details:
            output = tool_usage_details + "\n\n**Final Answer:** " + output

        # Prevent redundant messages in chat history
        if not chat_history or chat_history[-1].content != output:
            chat_history.append(HumanMessage(content=query))
            chat_history.append(AIMessage(content=output))

        return output, chat_history

    except Exception as e:
        error_message = f"Error executing agent: {str(e)}"
        print(error_message)
        return error_message, chat_history


# Example usage
if __name__ == "__main__":
    chat_history = []
    
    # Example queries to test the agent
    queries = [
        "What is the length of the word 'incomprehensibility'?",
        "What is the mean of these numbers: 10,15,20,25,30?",
        "What is the GCD of 48 and 18?",
        "Can you normalize this vector: [3, 4]?",
        "Remove the punctuation from this sentence: Hello, world! How are you doing today?"
    ]
    
    for query in queries:
        print(f"\n\nQuery: {query}")
        response, chat_history = run_agent(query, chat_history)
        print(f"Final Response: {response}")

/var/folders/vw/0x9ylhq57vq55_ssc_tyqhc00000gn/T/ipykernel_6035/1217889429.py:93: DeprecationWarning: callback_manager is deprecated. Please use callbacks instead.
  llm = ChatOllama(




Query: What is the length of the word 'incomprehensibility'?


> Entering new AgentExecutor chain...
Thought: I need to find out the length of the word
Action: get_word_length
Action Input: incomprehensibilityThought: I need to find out the length of the word
Action: get_word_length
Action Input: incomprehensibility19Thought: I've seen the result
Action: None (since we already have the answer)
Final Answer: The length of the word 'incomprehensibility' is 19.Thought: I've seen the result
Action: None (since we already have the answer)
Final Answer: The length of the word 'incomprehensibility' is 19.

> Finished chain.
Tools Available: ['get_word_length', 'get_mean_from_sequence', 'remove_punctuations', 'get_normalize_vector', 'get_gcd']

Raw result: {'input': "What is the length of the word 'incomprehensibility'?", 'chat_history': [], 'output': "The length of the word 'incomprehensibility' is 19.", 'intermediate_steps': [(AgentAction(tool='get_word_length', tool_input='incomprehensibi

In [37]:
# Add a quick debug function to test each tool directly
def test_tools():
    print("Testing tools directly:")
    print(f"get_word_length('incomprehensibility'): {get_word_length('incomprehensibility')}")
    # Test other tools similarly

test_tools()  # Call this before running the agent

Testing tools directly:
get_word_length('incomprehensibility'): 19


/var/folders/vw/0x9ylhq57vq55_ssc_tyqhc00000gn/T/ipykernel_6035/3054406739.py:4: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(f"get_word_length('incomprehensibility'): {get_word_length('incomprehensibility')}")
